In [2]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}
:dep log
:dep env_logger

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;
use log::{LevelFilter, info, warn};
use env_logger::Env;

use vawt::areofoil::Aerofoil;
use vawt::{VAWTSolver, VAWTSolution};

In [3]:
env_logger::init();
log::set_max_level(LevelFilter::Info);

In [4]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [5]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;
let aerofoil: &'static Aerofoil = Box::leak(Box::new(aerofoil));
let n = 90;
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(n)
    .epsilon(0.0001);

In [6]:
let solution = solver.tsr(3.25).solve_optimize_beta();

let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..2.0, -1f64..1f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.a(theta))), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.c_tan(theta))), 
        &BLUE
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta / PI, solution.beta(theta))),
        &GREEN
    ))?;
    Ok(())
})


-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
0.0π
 
 
 
0.5π
 
 
 
1.0π
 
 
 
1.5π
 
 
 
2.0π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="43,147 49,137 56,177 63,181 69,179 76,177 83,176 89,176 96,177 103,178 109,179 116,180 123,181 129,181 136,182 143,183 149,184 156,184 163,183 169,183 176,182 183,183 189,183 196,184 203,184 209,184 216,185 223,185 229,186 236,187 242,188 249,188 256,189 262,189 269,192 276,192 282,193 289,193 296,194 302,195 309,196 316,199 322,202 329,209 336,230 342,226 349,206 356,189 362,182 369,174 376,169 382,164 389,161 396,159 402,158 409,157 416,156 422,156 429,155 436,154 442,152 449,152 455,152 462,152 469,151 475,152 482,152 489,151 495,152 502,151 509,151 515,152 522,151 529,149 535,148 542,149 549,149 555,148 562,148 569,148 575,149 582,148 589,148 595,146 602,146 609,146 615,144 622,118 629,0 635,0 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="43,223 49,223 56,222 63,221 69,219 76,218 83,217 89,216 96,214 103,213 109,212 116,211 123,210 129,209 136,208 143,207 149,206 156,205 163,204 169,203 176,202 183,201 189,200 196,200 203,199 209,199 216,198 223,198 229,198 236,198 242,199 249,199 256,200 262,201 269,202 276,203 282,205 289,207 296,209 302,211 309,213 316,216 322,219 329,221 336,224 342,223 349,222 356,220 362,219 369,218 376,217 382,216 389,215 396,214 402,214 409,213 416,213 422,213 429,213 436,213 442,213 449,213 455,213 462,214 469,214 475,214 482,214 489,214 495,215 502,215 509,215 515,216 522,216 529,216 535,217 542,217 549,218 555,218 562,219 569,220 575,220 582,221 589,221 595,221 602,222 609,222 615,222 622,223 629,224 635,223 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="43,217 49,205 56,214 63,217 69,219 76,220 83,222 89,225 96,228 103,231 109,233 116,236 123,239 129,241 136,244 143,246 149,248 156,250 163,251 169,252 176,253 183,254 189,255 196,256 203,256 209,256 216,256 223,256 229,256 236,255 242,254 249,252 256,250 262,247 269,246 276,243 282,240 289,236 296,231 302,226 309,222 316,218 322,214 329,212 336,219 342,225 349,229 356,231 362,228 369,226 376,224 382,222 389,220 396,218 402,215 409,213 416,211 422,209 429,208 436,207 442,206 449,205 455,204 462,204 469,204 475,203 482,203 489,203 495,203 502,204 509,204 515,205 522,206 529,207 535,208 542,209 549,210 555,211 562,212 569,213 575,214 582,216 589,217 595,218 602,220 609,221 615,223 622,225 629,225 635,221 "/>

In [7]:
let tsr = Array::linspace(1.0,5.0,30);
let cps = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta(0.0).c_power()
    })
);
let cps_opt = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta_fn(|theta| solution.beta(theta)).c_power()
    })
);

In [8]:
evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 0f64..0.6)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps.iter().cloned()), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps_opt.iter().cloned()),
        &BLUE
    ))?;
    Ok(())
})

0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0